<div style="text-align: center;" align="center">
<img src="../../files/cara/apps/static/images/cara_logo_text_tight_margins.png" align="center" style="text-align: center;" /></div>
<p style="color: blue; font-weight: bold; font-size: x-large; text-align: center;" align="center">
Airborne Transmission of SARS-CoV-2
</p>

In [ ]:
import cara.apps

app = cara.apps.ExpertApplication()
app.widget